# Домашнее задание
* Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
* Реализовать самостоятельно логистическую регрессию
    * Обучить ее методом градиентного спуска
    * Методом nesterov momentum
    * Методом rmsprop
* В качестве dataset'а взять Iris, оставив 2 класса:
    * Iris Versicolor
    * Iris Virginica

In [1]:
import pandas as pd
import numpy as np
from math import ceil, floor

In [2]:
from sklearn.datasets import load_iris

In [3]:
print(load_iris()['DESCR'])

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [4]:
data = load_iris().data
feature_names = load_iris().feature_names

In [5]:
df = pd.DataFrame(data, columns=feature_names)

In [6]:
df['target'] = load_iris().target

**Так как нас интересуют только два класса: Versicolor, Virginica, то уберем те строчки, где target == 0 (это класс setosa).**

In [7]:
df = df[(df['target']==1) | (df['target']==2)].reset_index().drop(columns=['index'])
df['target'] -= 1

In [8]:
y = df['target']
X = df.drop(columns=['target'])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
X_train_tr = X_train.transpose()
y_train_tr = np.array(y_train).reshape(1,y_train.shape[0])
X_test_tr = X_test.transpose()
y_test_tr = np.array(y_test).reshape(1,y_test.shape[0])

**Реализуем самостоятельно логистическую регрессию и сравним результаты с LogisticRegression из sklearn.**

In [11]:
class LogRegression():
    
    def __init__(self, num_iterations=1000, learning_rate=0.01, method='gb', gamma=None, eps=None):
        """
        num_iterations - количество итераций цикла оптимизации
        learning_rate - скорость обучения
        method - метод оптимизации
        """
        self.num_iterations = num_iterations
        self.learning_rate = learning_rate
        self.method = method
        self.gamma = gamma
        self.eps = eps
        
    def sigmoid(self, x):

        s = 1.0 / (1.0 + np.exp(-x))

        return s

    def propagate(self, w, b, X, Y):
        """
        Реализует функцию стоимости и ее градиент.
        Принимает на вход:
        w - веса
        б - смещение, скаляр
        X - наши данные 
        Y - целевая функция

        Возвращает:
        cost - отрицательная логарифмическая вероятность для логистической регрессии
        dw - градиент w
        db - градиент b
        """
        
        number_of_features = X.shape[1]
        z = np.dot(w.T,X)+b
        A = self.sigmoid(z)
        #cost = -1.0/number_of_features*np.sum(Y*np.log(A)+(1.0-Y)*np.log(1.0-A))

        dw = 1.0/number_of_features*np.dot(X, (A-Y).T)
        db = 1.0/number_of_features*np.sum(A-Y)

        #cost = np.squeeze(cost)

        grads = {"dw": dw, 
                 "db":db}

        return grads

    def optimize(self, X, Y, method):
        """
        Этот метод оптимизирует w и b, различными алгоритмами. (Гр. спуск, nesterov momentum, rmsprop)

        Принимает на вход:
        w - веса
        б - смещение, скаляр
        X - наши данные
        Y - целевая функция
        num_iterations - количество итераций цикла оптимизации
        learning_rate - скорость обучения правила обновления градиентного спуска
        
        Возвращает:
        params - словарь, содержащий веса w и смещение b
        grads - словарь, содержащий градиенты весов и смещений
        """
        w = np.zeros((X.shape[0],1))
        b = 0

        costs = []
        
        if self.method == 'gb':

            for i in range(self.num_iterations):

                grads = self.propagate(w, b, X, Y)

                dw = grads["dw"]
                db = grads["db"]

                w = w - self.learning_rate*dw
                b = b - self.learning_rate*db
        
        elif self.method == 'nm':
            v_w = 0
            v_b = 0
            for i in range(self.num_iterations):

                grads = self.propagate(w, b, X, Y)

                dw = grads["dw"]
                db = grads["db"]

                v_w = self.gamma*v_w + self.learning_rate*(1-self.gamma)*dw
                
                v_b = self.gamma*v_b + self.learning_rate*(1-self.gamma)*db
                
                w = w - v_w
                b = b - v_b
                
        elif self.method == 'rmsp':
            EG_w = 0
            EG_b = 0
            for i in range(self.num_iterations):

                grads = self.propagate(w, b, X, Y)

                dw = grads["dw"]
                db = grads["db"]

                EG_w = self.gamma*EG_w + (1-self.gamma)*(dw/self.learning_rate)**2                
                EG_b = self.gamma*EG_b + (1-self.gamma)*(db/self.learning_rate)**2
                
                w = w - dw/(EG_w + self.eps)**0.5
                b = b - db/(EG_b + self.eps)**0.5

        grads = {"dw": dw, "db": db}
        params = {"w": w, "b": b}

        return params
        

    
    def predict(self, w, b, X):
        """
        Предсказывает будет значение 0 или 1 (граница 0.5).
        """

        y_prediction = np.zeros((1,X.shape[1]))
        w = w.reshape(X.shape[0],1)

        A = self.sigmoid(np.dot(w.T, X) + b)

        for i in range(A.shape[1]):
            if (A[:,i] > 0.5): 
                y_prediction[:, i] = 1
            elif (A[:,i] <= 0.5):
                y_prediction[:, i] = 0

        return y_prediction[0]
    
    def fit_predict(self, X_train, y_train, X_test):
        """
        Данный метод совершает оптимизацию и предсказывает значение y_test_predict.
        """
        
        parameters = self.optimize(X_train, y_train, self.method)

        w = parameters["w"]
        b = parameters["b"]

        y_test_predict = self.predict(w, b, X_test)

        return y_test_predict

**Обучим наши модели на разных способах оптимизации.**

In [12]:
clf_rmsp = LogRegression(num_iterations=1000, learning_rate=0.02, method='rmsp', gamma=0.8, eps=1e-6)
clf_gb = LogRegression(num_iterations=1000, learning_rate=0.02, method='gb')
clf_nm = LogRegression(num_iterations=1000, learning_rate=0.02, method='nm', gamma=0.9)

y_test_predict_rmsp = clf_rmsp.fit_predict(X_train_tr, y_train_tr, X_test_tr)
y_test_predict_gb = clf_gb.fit_predict(X_train_tr, y_train_tr, X_test_tr)
y_test_predict_nm = clf_nm.fit_predict(X_train_tr, y_train_tr, X_test_tr)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, f1_score, recall_score

In [14]:
LogisticRegression().fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_test_predict = clf.predict(X_test)

**Сравним наши результаты на различных метриках.**

In [16]:
clfs = [clf_rmsp, clf_gb, clf_nm, clf]
y_preds = [y_test_predict_rmsp, y_test_predict_gb, y_test_predict_nm, y_test_predict]

In [17]:
index = ['LG_rmsp', 'LG_gb', 'LG_nm', 'LG_sklearn']
metrics_columns = ['ROC_AUC', 'RECALL', 'ACCURACY', 'PRECISION', 'F1']
metrics_scores = np.zeros(20).reshape(4,5)

In [18]:
def metrics(y_test, y_pred, metric_type):
    
    if metric_type == 'ROC_AUC':
        return roc_auc_score(y_test, y_pred)
    
    elif metric_type == 'RECALL':
        return recall_score(y_test, y_pred)
    
    elif metric_type == 'ACCURACY':
        return accuracy_score(y_test, y_pred)
    
    elif metric_type == 'PRECISION':
        return precision_score(y_test, y_pred)
    
    elif metric_type == 'F1':
        return f1_score(y_test, y_pred)

In [19]:
for i in range(0, len(clfs)):  
    print('Классификатор: {}, считаем метрики...'.format(index[i]))
    for k in range(0, len(metrics_columns)):
        metrics_scores[i][k] = metrics(y_test, y_preds[i], metric_type=metrics_columns[k])
    print('Выполнено.')

Классификатор: LG_rmsp, считаем метрики...
Выполнено.
Классификатор: LG_gb, считаем метрики...
Выполнено.
Классификатор: LG_nm, считаем метрики...
Выполнено.
Классификатор: LG_sklearn, считаем метрики...
Выполнено.


In [20]:
metrics_df = pd.DataFrame(metrics_scores, index=index, columns=metrics_columns)
metrics_df

,ROC_AUC,RECALL,ACCURACY,PRECISION,F1
LG_rmsp,1.000000,1.0,1.00,1.000000,1.000000
LG_gb,0.944444,1.0,0.95,0.916667,0.956522
LG_nm,0.944444,1.0,0.95,0.916667,0.956522
LG_sklearn,0.944444,1.0,0.95,0.916667,0.956522


**Таким образом, на некоторых данных одна из наших моделей показала результат даже лучший, чем из коробки.**